In [1]:
from gzip import READ
from flask import Flask, request, render_template, redirect, url_for, flash, jsonify, send_from_directory, current_app
import requests
from string import Template
from flask_sqlalchemy import SQLAlchemy
from numpy import isnan
import sqlalchemy
import pandas as pd
pd.options.mode.chained_assignment = None
from pandas import DataFrame
from pandas.util import hash_pandas_object
import re
from flask_gtts import gtts
from config import GMAIL_PASSWORD, GMAIL_USERNAME, Config, S3_KEY, S3_SECRET, S3_BUCKET, SES_REGION_NAME, SES_EMAIL_SOURCE, GMAIL_USERNAME, GMAIL_PASSWORD, SERVER_NAME, SECRET_KEY
from flask_login import LoginManager
from models import LoginForm, RegistrationForm, PasswordResetForm, PasswordChangeForm, upload_file_to_s3, time_ago
from wtforms import validators
from wtforms.fields.html5 import EmailField
import email_validator
from passlib.hash import sha256_crypt
from flask_login import login_user, logout_user, login_required, current_user
import bcrypt
from datetime import datetime, timedelta
from dateutil import tz
from colour import Color
import re
import boto3, botocore
from django.utils.crypto import get_random_string
from werkzeug.utils import secure_filename
import os
import base64
import six
import uuid
import imghdr
import io
import pytz


# Not the entire world, just your best friends. 
application = Flask(__name__)
application.secret_key = application.config['SECRET_KEY']

application.config.from_object(Config)


#This is for users table
db = SQLAlchemy(application, engine_options={"pool_recycle": 1800})
db.init_app(application)

##Create SQL Engine Look at this: https://docs.sqlalchemy.org/en/14/core/pooling.html#pool-disconnects
engine = sqlalchemy.create_engine(application.config['SQLALCHEMY_DATABASE_URI'], pool_recycle=3600,)

In [ ]:

class User(db.Model):

    __tablename__ = 'users'
    email = db.Column(db.String, primary_key=True)
    id = db.Column(db.String)
    handle = db.Column(db.String)
    password = db.Column(db.String)
    authenticated = db.Column(db.Boolean, default=False)
    
    def is_active(self):
        """True, as all users are active."""
        return True
    
    def get_id(self):
        """Return the email address to satisfy Flask-Login's requirements."""
        return self.email

    def get_user_id(self):
        """Return the email address to satisfy Flask-Login's requirements."""
        return self.id   

    def get_user_handle(self):
        """Return the email address to satisfy Flask-Login's requirements."""
        return self.handle         
        
    def is_authenticated(self):
        """Return True if the user is authenticate#d."""
        return self.authenticated


In [ ]:
df = get_feed(8, None)
df = format_feed(df)

print(df)

In [61]:
def notify_mentionted_users(post_text, current_user):
    if '@' in post_text:
        #Get the list of users that are mentioned
        users = re.findall(r'@([a-zA-Z0-9_]+)', post_text)
        for user in users:
                #Get user_id of follow account
            with engine.connect() as connection:
                ResultProxy = connection.execute('''SELECT u.id 
                                                    FROM users u 
                                                    WHERE u.handle = %s; ''', (user))

            df = DataFrame(ResultProxy.fetchall())
            if len(df) > 0:
                df.columns = ResultProxy.keys()

                mentioned_user_id = df['id'][0]
                print(mentioned_user_id)


                #Get the post_id of the post
                with engine.connect() as connection:
                    ResultProxy = connection.execute('''SELECT p.post_id
                                                        FROM posts p 
                                                        WHERE p.user_id = %s 
                                                        ORDER BY p.post_id DESC
                                                        LIMIT 1; ''', (current_user))
                    df = DataFrame(ResultProxy.fetchall())
                    df.columns = ResultProxy.keys()

                post_id = df['post_id'][0]

                #Notify the user that someone mentioned them
                event_type_id = 3
                with engine.connect() as connection:
                    connection.execute('INSERT INTO notifications (user_id, triggered_by_user_id, event_type_id, reference_post_id) VALUES (%s, %s, %s, %s);',  (mentioned_user_id, current_user, event_type_id, post_id))
                    
                        
            

In [ ]:

post_text = "TEST - @Matthew have you ever seen this? https://www.youtube.com/watch?v=dQw4w9WgXcQ"

current_user = 8

notify_mentionted_users(post_text, current_user)
    

In [9]:
##Get notifications for this user
def get_notifications(user_id):
    with engine.connect() as connection:
            ResultProxy = connection.execute('''SELECT n.notification_id, n.creation_time, u.profile_photo, u.handle, n.event_type_id, n.reference_post_id, n.seen
                                                FROM notifications n
                                                LEFT JOIN users u ON u.id = n.triggered_by_user_id
                                                WHERE n.user_id = %s
                                                ORDER BY n.creation_time DESC
                                                LIMIT 25;
                                                ''', (user_id ))
            notifications = DataFrame(ResultProxy.fetchall())


    ## format the notifications
    if len(notifications.index) > 0:
        notifications.columns = ResultProxy.keys()

        #fill reference_post_id with 0s if NA
        notifications['reference_post_id'] = notifications['reference_post_id'].fillna(0)
        notifications['reference_post_id'] = notifications['reference_post_id'].astype(int)
        notifications['reference_post_id'] = notifications['reference_post_id'].astype(int)

        notifications['event_type_id'] = notifications['event_type_id'].astype(int)
        notifications['profile_photo'] = notifications['profile_photo'].fillna("")
        
        notifications['text'] = ''
        notifications['redirect'] = ''

        #Correct Timezone
        to_zone = tz.tzlocal()

        notifications['time_ago'] = ''
        #Create text for each notification
        for i in range(len(notifications.index)):
            if (notifications['event_type_id'][i] == 1):
                notifications['text'][i] = "now follows you"    
                notifications['redirect'][i] = "/@" + str(notifications['handle'][i])       

            if (notifications['event_type_id'][i] == 2):
                notifications['text'][i] = "replied to your post"
                notifications['reference_post_id'][i] = str(round(notifications['reference_post_id'][i], 0))
                notifications['redirect'][i] = "/post/" + str(notifications['reference_post_id'][i])

            if (notifications['event_type_id'][i] == 3):
                notifications['text'][i] = "mentioned you"
                notifications['reference_post_id'][i] = str(round(notifications['reference_post_id'][i], 0))
                notifications['redirect'][i] = "/post/" + str(notifications['reference_post_id'][i])    

            notifications['time_ago'][i] = time_ago(notifications['creation_time'][i].tz_localize('UTC').tz_convert(to_zone))
    
    #Sum count of unseen notifications
    unseen_count = 0
    for i in range(len(notifications.index)):
        if notifications['seen'][i] == 0:
            unseen_count += 1
    
    return notifications, unseen_count

In [10]:
post_id = 1000
user_id = 8

print(get_notifications(user_id))


(    notification_id       creation_time     profile_photo    handle  \
0               334 2022-07-10 16:06:38  9b2e4b78-b82.jpg     Kelly   
1               332 2022-07-09 18:18:46  9b2e4b78-b82.jpg     Kelly   
2               328 2022-07-09 10:36:44  9fbee8d6-74b.jpg   Matthew   
3               326 2022-07-08 13:45:19  9b2e4b78-b82.jpg     Kelly   
4               324 2022-07-08 03:43:50  9b2e4b78-b82.jpg     Kelly   
5               323 2022-07-08 03:43:42  9b2e4b78-b82.jpg     Kelly   
6               322 2022-07-08 03:05:54  e61b1134-85e.jpg      dave   
7               317 2022-07-08 03:01:42  e61b1134-85e.jpg      dave   
8               316 2022-07-08 02:33:59  e61b1134-85e.jpg      dave   
9               313 2022-07-07 21:56:46  e61b1134-85e.jpg      dave   
10              308 2022-07-06 22:44:14  e2bcd83e-d35.jpg       ben   
11              306 2022-06-30 11:48:39  daaea6b8-b45.jpg  courtney   
12              304 2022-06-27 18:44:30  9fbee8d6-74b.jpg   Matthew   
13   